In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models, similarities

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/haoyangwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/haoyangwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
data

,description,morn_comp_id,country,company,ind_code,mktcap_norm,related
morn_comp_id,,,,,,,
0C00003F1M,is a national insurance company. The principa...,0C00003F1M,United Arab Emirates,Al-Buhaira National Insurance Co,103,190.584,0C00003F0A 0C00008U58 0C00008FDU 0C00003F1L 0C...
0C00004ABT,is a commercial airline operator based in the...,0C00004ABT,United Arab Emirates,Abu Dhabi Aviation Co,310,835.585,0C00001HO2 0C0000C6UB 0C00001HOC 0C00001HOM 0C...
0C00001HOB,is a full-service commercial bank that operat...,0C00001HOB,United Arab Emirates,Abu Dhabi Commercial Bank,103,17099.551,0C00003F0G 0C00001RIY 0C00003F0D 0C00003F2D 0C...
0C00001HOP,is a banking corporation based in the United ...,0C00001HOP,United Arab Emirates,Abu Dhabi Islamic Bank,103,10956.582,0C00001TZM 0C00001RIY 0C00001HOB 0C00001HNZ 0C...
0C00003F1K,is engaged in the hospitality business. The c...,0C00003F1K,United Arab Emirates,Abu Dhabi National Hotels,102,2287.012,0C00003F27 0C00004ABS 0C0000CSZC 0C0000BDKU 0C...
...,...,...,...,...,...,...,...
0C00008JS9,is a Zimbabwe-based transport company. It is ...,0C00008JS9,Zimbabwe,Unifreight Africa Ltd,310,140.533,0C00008JSL 0C00008JSM 0C00008JS0 0C000018XU
0C00008JSN,is a clay brick manufacturer and distributor ...,0C00008JSN,Zimbabwe,Willdale Ltd,101,276.087,0C00008JRR 0C00008JSE 0C00008JSF
0C00008JSO,is a Zimbabwe-based investment holding compan...,0C00008JSO,Zimbabwe,Zb Financial Holdings Ltd,103,978.380,0C0000AWKT 0C00000NOV 0C00008JTS 0C00008JSR 0C...


In [5]:
data = pd.read_csv('new_description4.csv', header = None)
column = ['symbol', 'description', 'morn_comp_id', 'website', 'address', 'short_description',  'country', 'company', 'ind_code', 'mktcap_norm']
data.columns = column
data = data.drop(['symbol', 'website', 'address', 'short_description'], axis = 1)
data.index = data['morn_comp_id']
data.insert(data.shape[1], 'related', 0)
data = data.dropna()
ind_code_list = np.unique(data['ind_code'])
country_list = np.unique(data['country'])

In [41]:
data_idf = data.copy()

In [43]:
data['description'] = 1

In [44]:
data

,description,morn_comp_id,country,company,ind_code,mktcap_norm,related
morn_comp_id,,,,,,,
0C00003F1M,1,0C00003F1M,United Arab Emirates,Al-Buhaira National Insurance Co,103,190.584,0C00003F0A 0C00008U58 0C00008FDU 0C00003F1L 0C...
0C00004ABT,1,0C00004ABT,United Arab Emirates,Abu Dhabi Aviation Co,310,835.585,0C00001HO2 0C0000C6UB 0C00001HOC 0C00001HOM 0C...
0C00001HOB,1,0C00001HOB,United Arab Emirates,Abu Dhabi Commercial Bank,103,17099.551,0C00003F0G 0C00001RIY 0C00003F0D 0C00003F2D 0C...
0C00001HOP,1,0C00001HOP,United Arab Emirates,Abu Dhabi Islamic Bank,103,10956.582,0C00001TZM 0C00001RIY 0C00001HOB 0C00001HNZ 0C...
0C00003F1K,1,0C00003F1K,United Arab Emirates,Abu Dhabi National Hotels,102,2287.012,0C00003F27 0C00004ABS 0C0000CSZC 0C0000BDKU 0C...
...,...,...,...,...,...,...,...
0C00008JS9,1,0C00008JS9,Zimbabwe,Unifreight Africa Ltd,310,140.533,0C00008JSL 0C00008JSM 0C00008JS0 0C000018XU
0C00008JSN,1,0C00008JSN,Zimbabwe,Willdale Ltd,101,276.087,0C00008JRR 0C00008JSE 0C00008JSF
0C00008JSO,1,0C00008JSO,Zimbabwe,Zb Financial Holdings Ltd,103,978.380,0C0000AWKT 0C00000NOV 0C00008JTS 0C00008JSR 0C...


In [46]:
data_idf

,description,morn_comp_id,country,company,ind_code,mktcap_norm,related
morn_comp_id,,,,,,,
0C00003F1M,0,0C00003F1M,United Arab Emirates,Al-Buhaira National Insurance Co,103,190.584,0C00003F0A 0C00008U58 0C00008FDU 0C00003F1L 0C...
0C00004ABT,0,0C00004ABT,United Arab Emirates,Abu Dhabi Aviation Co,310,835.585,0C00001HO2 0C0000C6UB 0C00001HOC 0C00001HOM 0C...
0C00001HOB,0,0C00001HOB,United Arab Emirates,Abu Dhabi Commercial Bank,103,17099.551,0C00003F0G 0C00001RIY 0C00003F0D 0C00003F2D 0C...
0C00001HOP,0,0C00001HOP,United Arab Emirates,Abu Dhabi Islamic Bank,103,10956.582,0C00001TZM 0C00001RIY 0C00001HOB 0C00001HNZ 0C...
0C00003F1K,0,0C00003F1K,United Arab Emirates,Abu Dhabi National Hotels,102,2287.012,0C00003F27 0C00004ABS 0C0000CSZC 0C0000BDKU 0C...
...,...,...,...,...,...,...,...
0C00008JS9,0,0C00008JS9,Zimbabwe,Unifreight Africa Ltd,310,140.533,0C00008JSL 0C00008JSM 0C00008JS0 0C000018XU
0C00008JSN,0,0C00008JSN,Zimbabwe,Willdale Ltd,101,276.087,0C00008JRR 0C00008JSE 0C00008JSF
0C00008JSO,0,0C00008JSO,Zimbabwe,Zb Financial Holdings Ltd,103,978.380,0C0000AWKT 0C00000NOV 0C00008JTS 0C00008JSR 0C...


In [9]:
stop_words = set(stopwords.words('english'))
stop_words.add('ltd')
stop_words.add('inc')
stop_words.add('plc')
stop_words.add('company')

stemmer = PorterStemmer()

In [14]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [stemmer.stem(word) for word in words]
    final_text = words
    #final_text = ' '.join(words)
    return final_text

After the first edition of the code, we find that there are some companies having related companies with no common in their business but the similar name, like JPMorgan Chase Co and Chase packaging Corp. We tried to delete the company names in the descriptions.

In [35]:
string_list = []
for data_index, line in data.iterrows():
    string = line['description']
    string = string.replace(line['company'], '')
    string_list.append(string)
data['description'] = string_list

In [27]:
for indust in ind_code_list:
    indust_data = data[data['ind_code'] == indust]
    indust_data = indust_data.reset_index(drop = True)
    x = []
    for des in indust_data['description']:
        final_text = preprocess(des)
        x.append(final_text)
    dictionary = corpora.Dictionary(x)
    corpus = [dictionary.doc2bow(doc) for doc in x]
    tfidf = models.TfidfModel(corpus)
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = len(dictionary.keys()))
    count = 1
    for ind, test_text in enumerate(indust_data['description']):
        test_news_vec = [word for word in preprocess(test_text) if word not in stop_words]
        test_news_vec = tfidf[dictionary.doc2bow(test_news_vec)]
        sim = index[tfidf[test_news_vec]]
        similar_id = []
        current_id = indust_data.loc[ind, 'morn_comp_id']
        for indust_index, score in sorted(enumerate(sim), key=lambda item: -item[1])[0:11]:
            if indust_data.loc[indust_index, 'morn_comp_id'] == current_id:
                continue
            similar_id.append(indust_data.loc[indust_index, 'morn_comp_id'])
        if(len(similar_id) > 10):
            similar_id.pop()
        string = ' '.join(similar_id)
        data.loc[current_id, 'related'] = string
        if count % 1000 == 0:
            print("This is the " + str(count) + "th update.")
        count += 1
    print("Industry " + str(indust) + " has completed!")

This is the 1000th update.
This is the 2000th update.
This is the 3000th update.
This is the 4000th update.
This is the 5000th update.
Industry 101 has completed!
This is the 1000th update.
This is the 2000th update.
This is the 3000th update.
This is the 4000th update.
This is the 5000th update.
Industry 102 has completed!
This is the 1000th update.
This is the 2000th update.
This is the 3000th update.
This is the 4000th update.
This is the 5000th update.
Industry 103 has completed!
This is the 1000th update.
This is the 2000th update.
Industry 104 has completed!
This is the 1000th update.
This is the 2000th update.
Industry 205 has completed!
This is the 1000th update.
This is the 2000th update.
This is the 3000th update.
This is the 4000th update.
Industry 206 has completed!
Industry 207 has completed!
This is the 1000th update.
This is the 2000th update.
Industry 308 has completed!
This is the 1000th update.
Industry 309 has completed!
This is the 1000th update.
This is the 2000th 

In [31]:
data

,description,morn_comp_id,country,company,ind_code,mktcap_norm,related
morn_comp_id,,,,,,,
0C00003F1M,is a national insurance company. The principa...,0C00003F1M,United Arab Emirates,Al-Buhaira National Insurance Co,103,190.584,0C00003F0A 0C00008U58 0C00008FDU 0C00003F1L 0C...
0C00004ABT,is a commercial airline operator based in the...,0C00004ABT,United Arab Emirates,Abu Dhabi Aviation Co,310,835.585,0C00001HO2 0C0000C6UB 0C00001HOC 0C00001HOM 0C...
0C00001HOB,is a full-service commercial bank that operat...,0C00001HOB,United Arab Emirates,Abu Dhabi Commercial Bank,103,17099.551,0C00003F0G 0C00001RIY 0C00003F0D 0C00003F2D 0C...
0C00001HOP,is a banking corporation based in the United ...,0C00001HOP,United Arab Emirates,Abu Dhabi Islamic Bank,103,10956.582,0C00001TZM 0C00001RIY 0C00001HOB 0C00001HNZ 0C...
0C00003F1K,is engaged in the hospitality business. The c...,0C00003F1K,United Arab Emirates,Abu Dhabi National Hotels,102,2287.012,0C00003F27 0C00004ABS 0C0000CSZC 0C0000BDKU 0C...
...,...,...,...,...,...,...,...
0C00008JS9,is a Zimbabwe-based transport company. It is ...,0C00008JS9,Zimbabwe,Unifreight Africa Ltd,310,140.533,0C00008JSL 0C00008JSM 0C00008JS0 0C000018XU
0C00008JSN,is a clay brick manufacturer and distributor ...,0C00008JSN,Zimbabwe,Willdale Ltd,101,276.087,0C00008JRR 0C00008JSE 0C00008JSF
0C00008JSO,is a Zimbabwe-based investment holding compan...,0C00008JSO,Zimbabwe,Zb Financial Holdings Ltd,103,978.380,0C0000AWKT 0C00000NOV 0C00008JTS 0C00008JSR 0C...


We then divided the companies with their countries.

In [29]:
for indust in ind_code_list:
    indust_data_total = data[data['ind_code'] == indust]
    for country in country_list:
        indust_data = indust_data_total[indust_data_total['country'] == country]
        indust_data = indust_data.reset_index(drop = True)
        x = []
        for des in indust_data['description']:
            final_text = preprocess(des)
            x.append(final_text)
        dictionary = corpora.Dictionary(x)
        corpus = [dictionary.doc2bow(doc) for doc in x]
        tfidf = models.TfidfModel(corpus)
        index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = len(dictionary.keys()))
        count = 1
        for ind, test_text in enumerate(indust_data['description']):
            test_news_vec = [word for word in preprocess(test_text) if word not in stop_words]
            test_news_vec = tfidf[dictionary.doc2bow(test_news_vec)]
            sim = index[tfidf[test_news_vec]]
            similar_id = []
            current_id = indust_data.loc[ind, 'morn_comp_id']
            if len(sim) > 10:
                num = 11
            else:
                num = len(sim)
            for indust_index, score in sorted(enumerate(sim), key=lambda item: -item[1])[0:num]:
                if indust_data.loc[indust_index, 'morn_comp_id'] == current_id:
                    continue
                similar_id.append(indust_data.loc[indust_index, 'morn_comp_id'])
            if(len(similar_id) > 10):
                similar_id.pop()
            string = ' '.join(similar_id)
            data.loc[current_id, 'related'] = string
            if count % 1000 == 0:
                print("This is the " + str(count) + "th update.")
            count += 1
        print("Industry " + str(indust) + ", country " + str(country) + " has completed!")

Industry 101, country Argentina has completed!
Industry 101, country Australia has completed!
Industry 101, country Austria has completed!
Industry 101, country Bahrain has completed!
Industry 101, country Bangladesh has completed!
Industry 101, country Belgium has completed!
Industry 101, country Bosnia and Herzegovina has completed!
Industry 101, country Brazil has completed!
Industry 101, country Bulgaria has completed!
This is the 1000th update.
Industry 101, country Canada has completed!
Industry 101, country Chile has completed!
Industry 101, country China has completed!
Industry 101, country Columbia has completed!
Industry 101, country Cote d'Ivoire has completed!
Industry 101, country Croatia has completed!
Industry 101, country Cyprus has completed!
Industry 101, country Czech Republic has completed!
Industry 101, country Denmark has completed!
Industry 101, country Ecuador has completed!
Industry 101, country Egypt has completed!
Industry 101, country Estonia has completed!


In [30]:
data.loc['0C00000BMT', 'related']

'0C000006U3 0C000009MU 0C0000934D 0C000055CG 0C000007AN 0C00000ABK 0C0000B0A6 0C0000C75J 0C00000CU1 0C00000KP8'

In [28]:
sim

array([0.], dtype=float32)

In [27]:
data[data['country'] == 'China']

,description,morn_comp_id,country,ind_code,mktcap_norm,related
morn_comp_id,,,,,,
0C00000OT1,Shanghai Pudong Development Bank Co Ltd provid...,0C00000OT1,China,103,28824.197,0C00001RHH 0C00000P70 0C00000TZT 0C0000AL6U 0C...
0C00000OT7,Guangzhou Baiyun International Airport Co Ltd ...,0C00000OT7,China,310,3310.347,0C00000P9C 0C00000PA8 0C00000OII 0C00000Q7S 0C...
0C00000OTB,Dongfeng Automobile Co Ltd is engaged in the m...,0C00000OTB,China,102,1652.892,0C00000ODJ 0C00000P64 0C00000P42 0C00000ONO 0C...
0C00000OTC,China World Trade Center Co Ltd is engaged in ...,0C00000OTC,China,104,2642.900,0C000051S4 0C00000OEI 0C00000OM1 0C00000OS6 0C...
0C00000OTD,Beijing Capital Eco-Environment Protection Gro...,0C00000OTD,China,310,2844.675,0C00000PAO 0C0000BD18 0C00000P3L 0C000068ZI 0C...
...,...,...,...,...,...,...
0C0000DE30,Zhejiang Chemsyn Pharm Co Ltd is engaged in re...,0C0000DE30,China,206,187.791,0C0000BDUW 0C0000CUL0 0C0000B971 0C0000B87D 0C...
0C0000DDZE,Chengdu Sino Microelectronics Tech Co Ltd is e...,0C0000DDZE,China,311,2123.855,0C00000P0H 0C0000CV0G 0C0000C1MT 0C0000CCIY 0C...
0C0000DE2Z,Wafer Works (Shanghai) Co Ltd is engaged in th...,0C0000DE2Z,China,311,1857.575,0C00009CLI 0C00000OQB 0C00000QAL 0C00006EKU 0C...


In [13]:
len(sim)

6324

In [12]:
data.to_csv('related10.csv')